## Black Scholes Merton Model

여기서는 BSM Model의 이산 시간 버전으로 시작한다. 잘 알려진 바와 같이 이 공식에서 option hedging과 가격 결정의 문제는 순차적 위험 최소화에 해당한다. 해결해야 할 주요 문제는 옵션에서 위험을 어떻게 정의하느냐이다. 이 부분에서 Föllmer와 Schweizer(1989), Schweizer(1995), Cerný와 Kallsen(2007)의 연구에서 개척된 국지적 위험 최소화 접근법을 따른다. 비슷한 방법이 물리학자 Potters et al.(2001)에 의해 개발됐다. Petrelli et al.(2010)의 작업도 참고하자. 우리는 Grau(2007)에서 제안된 버전의 접근법을 사용한다.

이 접근법에서 만기 시점의 최종 주가 $S_T$에 의존하는 만기 시의 최종 수익 $H_T(S_T)$인 European Option의 매도자 입장을 취한다. 옵션을 헤징하고자 매도자는 매도 수익금을 주식 $S_t$와 무위험 은행 예금 $B_t$로 구성된 복제 포트폴리오 $\Pi_t$를 설정한다. hedge portfolio의 가치는 어떤 시점 $t \geq T$에서도 다음과 같이 된다.

$$\Pi_t = u_t S_t + B_t$$

여기서 $u_t$는 옵션 위험을 헤징하고자 취해진 시점 $t$의 주식 포지션이다.